# exercise 8

In [1]:
import sklearn 
assert sklearn.__version__ >= '0.20'
import os
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [3]:
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(mnist.data, mnist.target, test_size=1000, random_state=2021)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=1000, random_state=2021)

In [4]:
rf_clf=RandomForestClassifier(n_estimators=10, random_state=2021)
ext_clf=ExtraTreesClassifier(n_estimators=10, random_state=2021)
svm_clf=LinearSVC(max_iter=100, random_state=2021)
knn_clf=KNeighborsClassifier(n_neighbors=5)

In [5]:
from sklearn.metrics import accuracy_score

In [26]:
classifiers=[rf_clf, ext_clf, svm_clf,knn_clf]

for model in classifiers:
    model.fit(X_train, y_train)

/Users/nam-ilyang/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [7]:
for model in classifiers:
    y_pred=model.predict(X_val)
    print(f'accuracy of {model} : ', accuracy_score(y_val, y_pred))

accuracy of RandomForestClassifier(n_estimators=10, random_state=2021) :  0.956
accuracy of ExtraTreesClassifier(n_estimators=10, random_state=2021) :  0.961
accuracy of LinearSVC(max_iter=100, random_state=2021) :  0.863
accuracy of KNeighborsClassifier() :  0.971


In [8]:
from sklearn.ensemble import VotingClassifier

classifier_model = [('RandomForest', rf_clf),
                   ('extraTreeForest', ext_clf),
                   ('LinearSVM', svm_clf),
                   ('KNNeighbor',knn_clf)]

In [9]:
voting_clf=VotingClassifier(classifier_model)

In [10]:
voting_clf.fit(X_train, y_train)

/Users/nam-ilyang/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


VotingClassifier(estimators=[('RandomForest',
                              RandomForestClassifier(n_estimators=10,
                                                     random_state=2021)),
                             ('extraTreeForest',
                              ExtraTreesClassifier(n_estimators=10,
                                                   random_state=2021)),
                             ('LinearSVM',
                              LinearSVC(max_iter=100, random_state=2021)),
                             ('KNNeighbor', KNeighborsClassifier())])

In [11]:
voting_clf.score(X_val, y_val)

0.973

In [12]:
classifier_1=[('RandomForest', rf_clf),('extraTreeForest', ext_clf),('LinearSVM', svm_clf)]
classifier_2=[('RandomForest', rf_clf),('extraTreeForest', ext_clf),('KNNeighbor',knn_clf)]
classifier_3=[('RandomForest', rf_clf),('LinearSVM', svm_clf),('KNNeighbor',knn_clf)]
classifier_4=[('extraTreeForest', ext_clf),('LinearSVM', svm_clf),('KNNeighbor',knn_clf)]

voting_clf_1=VotingClassifier(classifier_1)
voting_clf_2=VotingClassifier(classifier_2)
voting_clf_3=VotingClassifier(classifier_3)
voting_clf_4=VotingClassifier(classifier_4)

voting_clf_1.fit(X_train,y_train)
voting_clf_2.fit(X_train,y_train)
voting_clf_3.fit(X_train,y_train)
voting_clf_4.fit(X_train,y_train)

/Users/nam-ilyang/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Users/nam-ilyang/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Users/nam-ilyang/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


VotingClassifier(estimators=[('extraTreeForest',
                              ExtraTreesClassifier(n_estimators=10,
                                                   random_state=2021)),
                             ('LinearSVM',
                              LinearSVC(max_iter=100, random_state=2021)),
                             ('KNNeighbor', KNeighborsClassifier())])

In [13]:
print(voting_clf_1.score(X_val, y_val))
print(voting_clf_2.score(X_val, y_val))
print(voting_clf_3.score(X_val, y_val))
print(voting_clf_4.score(X_val, y_val))

0.965
0.972
0.973
0.97


In [14]:
voting_clf_2.voting='soft'

In [15]:
voting_clf_2.score(X_val, y_val)

0.976

In [16]:
voting_clf_2.score(X_test, y_test)

0.97

# exercise 9

In [17]:
X_val_pred=np.empty((len(X_val), len(classifiers)),dtype=np.float32)

In [18]:
for i, model in enumerate(classifiers):
    X_val_pred[:,i]=model.predict(X_val)

In [19]:
rf_blender=RandomForestClassifier(n_estimators=100,oob_score=True,random_state=2021)
rf_blender.fit(X_val_pred, y_val)

RandomForestClassifier(oob_score=True, random_state=2021)

In [20]:
rf_blender.oob_score_

0.97

In [22]:
X_test_pred=np.empty((len(X_test), len(classifiers)),dtype=np.float32)

for i, model in enumerate(classifiers):
    X_test_pred[:,i]=model.predict(X_test)

In [24]:
y_pred=rf_blender.predict(X_test_pred)

In [25]:
accuracy_score(y_test,y_pred)
#왠만한 개별적인 분류기 보다 나은 결과를 보인다

0.96